In [1]:
import ipywidgets as widgets
from ipywidgets import Image, interact, interactive, fixed, interact_manual, interactive_output, IntProgress
from ipywidgets import Button, Box, BoundedFloatText, Text, Textarea, Dropdown, Label, IntSlider, SelectMultiple, Accordion
from ipywidgets import AppLayout, Layout, FloatSlider, IntText, VBox, HBox, Output
from IPython.display import display

from concurrent.futures import wait
import time
from time import sleep

from IPython.display import display
import plotly.graph_objects as go
from plotly.subplots import make_subplots

%matplotlib ipympl
import matplotlib.pyplot as plt
import numpy as np

# import Qiskit library
from qiskit import *
from qiskit_aer import QasmSimulator

In [2]:
def progress_bar(i, total):
  sim_progress.value = i/total
  percent_complete = round((i/total)*100)
  sim_progress.description = '{} %'.format(percent_complete)

  spbar_colors = ['DarkRed', 'FireBrick', 'Crimson', 'Red', 'OrangeRed', 'DarkOrange', 'Orange', 'Gold', 'Yellow', 'PaleGreen', 'SpringGreen']
  n_colors = len(spbar_colors)

  for nc in range(n_colors-2):
    if (0.108*nc) <= sim_progress.value < (0.108*(nc+1)):
      sim_progress.style = style={'bar_color': spbar_colors[nc]}

  if 0.972 <= sim_progress.value < 0.99:
    sim_progress.style = style={'bar_color': spbar_colors[n_colors-2]}

  elif 0.99 <= sim_progress.value <= 1.0:
      sim_progress.style = style={'bar_color': spbar_colors[n_colors-1]}

sim_progress = widgets.FloatProgress(min=0, max=1, bar_style='', orientation='horizontal')
sim_prgss_label = widgets.Label(value='Simulation Progress:')
sim_prgs_bar = widgets.Box([sim_prgss_label, sim_progress])

In [3]:
# Trotter propagation in transverse-field Ising model
def get_spin_measurements(J, B, dt, total_time, percentage0_0, percentage1_0):
    # J, B, and dt are physical parameters (atomic units)
    count_00, count_01, count_10, count_11 = np.array([]), np.array([]), np.array([]), np.array([])
    i = 0
    while i <= total_time + dt:
        progress_bar(i, total_time + dt) # updating progress bar

        circ = QuantumCircuit(2, 2) # 2 particles, IC both at 0
        percentage0_1 = 1-percentage0_0 # superposition on the 0th qubit 
        percentage1_1 = 1-percentage1_0 # superposition on the 1st qubit 
        if (percentage0_0 != 1): # prevents division by zero
            circ.ry(np.arctan(percentage0_0/percentage0_1) * 2, 0)
        if (percentage1_0 != 1):
            circ.ry(np.arctan(percentage1_0/percentage1_1) * 2, 1)
        #plot_bloch_multivector(Statevector(circ)) # to check the superposition makes sense

        for t in np.linspace(0, i, int(i/dt)):
            circ.rx(-2*dt*B, 0) # Transverse-field propagation of spin 0
            circ.rx(-2*dt*B, 1) # Transverse-field propagation of spin 1
            circ.cx(0, 1)       # Exchange-coupling time propagation (1)
            circ.rz(-2*dt*J, 1) # (2)
            circ.cx(0, 1)       # (3)
        circ.measure(range(2), range(2)) # Measure both spins

        backend = QasmSimulator()
        circ_compiled = transpile(circ, backend)     # Transpile the quantum circuit to low-level QASM instructions
        job_sim = backend.run(circ_compiled, shots=1024)     # Execute the circuit on the Qasm simulator, repeating 1024 times
        result_sim = job_sim.result() # Grab the results from the job
        counts = result_sim.get_counts(circ_compiled) # Get counts

        if '00' in counts:  count_00 = np.append(count_00, counts['00'])
        else:               count_00 = np.append(count_00, 0)

        if '01' in counts:  count_01 = np.append(count_01, counts['01'])
        else:               count_01 = np.append(count_01, 0)

        if '10' in counts:  count_10 = np.append(count_10, counts['10'])
        else:               count_10 = np.append(count_10, 0)

        if '11' in counts:  count_11 = np.append(count_11, counts['11'])
        else:               count_11 = np.append(count_11, 0)
        i = i + dt
    return count_00, count_01, count_10, count_11

In [4]:
def simulate_TFIM(b):
    sim_progress.value, sim_progress.description = 0, ''
    J, B, dt, total_time, percentage0_0, percentage1_0 = J_slider.value, B_slider.value, time_step.value, sim_time.value, qubit0.value/100, qubit1.value/100
    count_00, count_01, count_10, count_11 = get_spin_measurements(J, B, dt, total_time, percentage0_0, percentage1_0)
    t = np.linspace(0, total_time, int(total_time/dt) + 1)

    # Display bar graph 
    fig = go.FigureWidget()
    bar = fig.add_bar(x=['Counts 00', 'Counts 01', 'Counts 10', 'Counts 11'], marker_color=['red', 'orange', 'green', 'blue'])
    a, b, c, d = np.max(count_00), np.max(count_01), np.max(count_10), np.max(count_11)
    fig.layout = dict(yaxis=dict(range=[0,np.max([a,b,c,d])+10], title='Counts'), 
                    height=300, width=500, showlegend=False, margin=dict(l=0, r=0, t=0, b=0))

    # Counts vs. time 
    fig2 = go.FigureWidget(make_subplots(rows=4, cols=1, shared_yaxes=True, shared_xaxes=True))
    fig2.add_trace(go.Scatter(x=t, y=count_00, mode='lines', name='Counts 00'), row=1, col=1)
    fig2.add_trace(go.Scatter(x=t, y=count_01, mode='lines', name='Counts 01'), row=2, col=1)
    fig2.add_trace(go.Scatter(x=t, y=count_10, mode='lines', name='Counts 10'), row=3, col=1)
    fig2.add_trace(go.Scatter(x=t, y=count_11, mode='lines', name='Counts 11'), row=4, col=1)
    fig2.add_trace(go.Scatter(x=[0], y=[count_00[0]], mode='markers', marker=dict(color='black', size=8), showlegend = False), row=1, col=1)
    fig2.add_trace(go.Scatter(x=[0], y=[count_01[0]], mode='markers', marker=dict(color='black', size=8), showlegend = False), row=2, col=1)
    fig2.add_trace(go.Scatter(x=[0], y=[count_10[0]], mode='markers', marker=dict(color='black', size=8), showlegend = False), row=3, col=1)
    fig2.add_trace(go.Scatter(x=[0], y=[count_11[0]], mode='markers', marker=dict(color='black', size=8), showlegend = False), row=4, col=1)

    fig2.update_layout(title_text="Counts vs. Time", autosize=True, margin=dict(l=20, r=20, t=50, b=50),
                        title_x=0.5,  # Set the title's horizontal position to the center
                        title_xanchor='center')  # Center the title horizontally
    fig2.update_xaxes(title_text="Time", range=[0-dt, np.max(t)+dt], row=4, col=1)
    fig2.update_yaxes(range=[-100, np.max([np.max(count_00), np.max(count_01), np.max(count_10), np.max(count_11)])+50])

    @interact(t=(0, total_time, dt))
    def update(t=0):
        i = int(t/dt)
        with fig.batch_update():
            fig.data[0].y=[count_00[i], count_01[i], count_10[i], count_11[i]]
        with fig2.batch_update():
            fig2.data[4].x, fig2.data[5].x, fig2.data[6].x, fig2.data[7].x = [t], [t], [t], [t]
            fig2.data[4].y, fig2.data[5].y, fig2.data[6].y, fig2.data[7].y = [count_00[i]], [count_01[i]], [count_10[i]], [count_11[i]]
            
    graph = widgets.VBox([fig])
    graph2 = widgets.VBox([fig2])
    graphs = widgets.HBox([graph, graph2])
    display(graphs)

In [5]:
def reset(b):
    J_slider.value, B_slider.value, time_step.value, sim_time.value, qubit0.value, qubit1.value = 0,0,0.01,0,0,0
    sim_progress.value, sim_progress.description = 0, ''
    # figure_output.clear_output()

In [ ]:
slider_layout = widgets.Layout(width='400px')
style = {'description_width': 'initial', 'width': 'auto'}

qubit0 = widgets.IntSlider(description='% of Qubit 0:', min=0, max=100, disabled=False, layout=slider_layout)
qubit1 = widgets.IntSlider(description='% of Qubit 1:', min=0, max=100, disabled=False, layout=slider_layout)
superposition = widgets.VBox([qubit0, qubit1])
superposition_label = widgets.Label(value='Choose % in 0 superposition of qubits 0 and 1:')

J_slider = widgets.FloatSlider(description='Coupling strength, J:', min=0, max=20, style=style, layout=slider_layout)
B_slider = widgets.FloatSlider(description='Transverse magnetic field strength, B:', min=0, max=20, style=style, layout=slider_layout)
time_step = widgets.FloatSlider(description='Time step (sec), dt:', value=0.000, min=0.01, max=0.1, step=0.001, disabled=False, style=style, layout=slider_layout)
sim_time = widgets.FloatSlider(description='Simulation time (sec): t', min=0, max=6, style=style, layout=slider_layout)

reset_bttn = widgets.Button(description='Reset', disabled=False, button_style='', tooltip='Click me')
simulate_bttn = widgets.Button(description='Simulate', disabled=False, button_style='', tooltip='Click me', icon='check')
buttons = widgets.HBox([simulate_bttn, reset_bttn])
parameters = widgets.VBox([superposition_label, superposition, J_slider, B_slider, time_step, sim_time, buttons, sim_prgs_bar], layout={'width': '800px'})

output = widgets.Output()
simulate_bttn.on_click(simulate_TFIM)
reset_bttn.on_click(reset)
game_page = widgets.VBox([parameters, output])
display(game_page)

'''  
with figure_output:
    display(counts_figures)
  figure_output.clear_output(wait=True)
'''

'  \nwith figure_output:\n    display(counts_figures)\n  figure_output.clear_output(wait=True)\n'

interactive(children=(FloatSlider(value=0.0, description='t', max=0.8, step=0.01), Output()), _dom_classes=('w…

    'data': [{'marker': {'color': ['red', 'orange', 'green', 'blu…

interactive(children=(FloatSlider(value=0.0, description='t', max=0.8, step=0.01), Output()), _dom_classes=('w…

    'data': [{'marker': {'color': ['red', 'orange', 'green', 'blu…

Things to Finish:
- Make sure the graphs clear
- Write up and make it look nice

In [7]:
from IPython.display import display, Math, Latex

In [8]:
display(Math(r'H=-J\displaystyle\sum_{i,j} s_{i}s_{j}-B\sum_{i}^{ }s_{i}'))
display(Math(r'H=-J\displaystyle\sum_{i,j}^{ }s_{i}^{z}s_{j}^{z}-B\sum_{i}^{ }s_{i}^{x} '))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [9]:
title = widgets.HTML(value=f"""<h1 style="text-align: center"> Ising Model </h1>""")

label_layout = widgets.Layout(width='600px', height='flex', align_items='flex-end')
text = '''Many quantum mechanical systems can be described by a few relevant “states”. For example, an atom in its ground 
        state (“state 1”) can be driven by laser light to an excited state (“state 2”). In this case, all other states of the atom can be neglected. '''
game_description_text = f""" 
<div style="text-align: justify;">
    {text}
</div> """

game_description = widgets.HTML(value=game_description_text, layout=label_layout)
transition_section = widgets.HTML(value=f"""<hr>""")
info_pg = widgets.VBox([title, transition_section, game_description, transition_section])
info_pg.align = "bottom"

In [10]:
full_page = widgets.VBox([info_pg, game_page])
display(full_page)